In [ ]:
# 2025-6-29 run it on "python 3" base env on ins-gl-pt-gpu24-2422a2a-gpuprof-3env-j4-1b with L4 GPU
# on amlo-yieldoptim, generated new tar files with latents and caption emb, then train model with the new tar files

# 2025-6-2 run it on "python 3" base env on ins-gl-pt-gpu24-2422a2a-gpuprof-3env-j4-1b with L4 GPU

# 2025-5-20 run it on "python 3" base env on ins-gl-pt-gpu24-efe8b01-j4-0env-pdp with L4 GPU

# ====================================
# 2025-5-13 run it on "python 3" base env on ins-gl-pt-gpu24-2c94136-3env-j4-l4-test-1 with 1 L4 GPU
# run on a single GPU notebook cell you can point the same script directly

# steps of updating data to remvove mask channel:
# 1. run this script (/Users/jinxuding/Downloads/CV/cubediff/implementation/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py) 
#   to generate the new data
# 2. run this shell script (/Users/jinxuding/Downloads/CV/cubediff/implementation/llm-cv-pano-cubediff/cl/training/create_tar.sh) 
#   to create the tar file


# download data

In [ ]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/llm-cv-pano-cubediff"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

In [4]:
# 2025-5-20 copied data from "merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/data"
# so no need to download and generate tar files.
# # after installing packages
# !python /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py \
#       --out /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny 
#       --skip_download \
#       --skip_convert

[2025-05-20 16:51:40,044] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
INFO:cl.data.polyhaven.cubemap_builder:PyTorch GPU acceleration available: NVIDIA L4
INFO:cl.data.polyhaven.cubemap_builder:Using 94 CPU cores for parallel processing
INFO:cl.data.polyhaven.api_client:Fetching list of HDRIs from Polyhaven API (limit: 700)...
ERROR:cl.data.polyhaven.api_client:Error accessing Polyhaven API: HTTPSConnectionPool(host='api.polyhaven.com', port=443): Max retries exceeded with url: /assets (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7dfb5c88cca0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))
INFO:cl.data.polyhaven.api_client:Downloading 15 new HDRIs out of 15 total
^C
Process ForkProcess-31:
Process ForkProcess-29:
Process ForkProcess-22:
Process ForkProcess-25:
Process ForkProcess-24:
Process ForkProcess-32:
Process ForkProcess-30:
Process ForkProcess-21:
Process ForkProcess

# set up env 

# Step 1: build dataset from exr files (with testing)

In [2]:
!sudo mount -o remount,size=64G /dev/shm

In [3]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         193G   70G  124G  36% /
tmpfs            64M     0   64M   0% /dev
shm              64G     0   64G   0% /dev/shm
/dev/nvme0n2    196G   29G  168G  15% /home/jupyter
/dev/nvme0n1p1  193G   70G  124G  36% /usr/local/nvidia/bin


In [4]:
# 2025-6-14 build latent (.pt, shape [1, 6, 64, 64]), panorama, faces, captions (and embedding) and data validation report
!time python /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/polyhaven/build_tiny_set.py \
    --exr_dir "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny/raw/erp" \
    --output_dir "/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_13_new_pt" \
    --cpu_workers 12 \
    --gpu_batch_size 64 \
    --split_ratio 0.9  > build_tiny_set_log_bs64_2025_6_29.txt

# 2025-6-29 64 bs on 1 L4
# real	7m54.179s
# user	111m58.960s
# sys	8m5.171s

2025-06-30 04:20:44,763 - INFO - PyTorch GPU acceleration available: NVIDIA L4
Loading pipeline components...: 100%|█████████████| 7/7 [00:21<00:00,  3.01s/it]
2025-06-30 04:21:14,014 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:14,829 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,029 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,106 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,118 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,121 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,199 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,208 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,224 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,240 - INFO - PyTorch GPU acceleration available: NVIDIA L4
2025-06-30 04:21:15,255 - INFO - PyTorch GPU accel

# Step 2: Create WebDataset files (tar files for train and val) (with testing)  

In [5]:
# !chmod +x create_tar.sh  
!./create_tar.sh \
    --data_root /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_13_new_pt \
    --workers 8 > create_tar_log_2025_6_29.txt

/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")


# install packages

In [1]:
!pip install torch==2.1.2 torchvision==0.16.2 "peft>=0.8.2" \
     opencv-python==4.8.1.78 opencv-contrib-python==4.8.1.78 matplotlib==3.8.2 tqdm==4.66.1 einops==0.7.0 \
     xformers requests pillow openexr opencv-python-headless wandb \
     bitsandbytes webdataset timm diffusers huggingface_hub accelerate deepspeed imageio \
     -i https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple

!pip uninstall torch-xla -y

Looking in indexes: https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/simple
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/03/f1/13137340776dd5d5bcfd2574c9c6dfcc7618285035cd77240496e5c1a79b/torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/da/ae/76bd3682465730dea7be21f36a8160a911a470de6f26228904f222e7fefe/torchvision-0.16.2-cp310-cp310-manylinux1_x86_64.whl (6.8 MB)
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/68/85/8e6ea3d1089f2b6de3c1cd34bbbd7560912af9d34b057be3b8b8fefe1da3/peft-0.15.2-py3-none-any.whl (411 kB)
  Using cached https://artifactory.code.bestbuy.com/artifactory/api/pypi/pypi/packages/packages/b7/8a/b2f7e1a434d56bf1d7570fc5941ace0847404e1032d7f1f0b8fed896568d/opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (61.7 MB)
  Using cac

In [2]:
!pip list | grep "acc\|torch\|diff\|trans"

accelerate                               1.8.1
diffusers                                0.34.0
torch                                    2.1.2
torchvision                              0.16.2
transformers                             4.53.0


In [3]:
import OpenEXR

In [4]:
!pip show OpenEXR

Name: OpenEXR
Version: 3.3.4
Summary: Python bindings for the OpenEXR image file format
Home-page: https://openexr.com
Author: 
Author-email: Contributors to the OpenEXR project <info@openexr.com>
License: 
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy
Required-by: 


# create tar files

In [9]:
# !chmod +x create_tar.sh # 
!time ./create_tar.sh \
    --data_root /home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/data/dataspace/polyhaven_tiny_13_new_pt \
    --workers 8 > create_tar_log_2025_6_29.txt

/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
/opt/conda/lib/python3.10/site-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")

real	0m52.877s
user	0m45.080s
sys	0m40.177s


# config shm

In [4]:
# allocate more memory for shm
!sudo mount -o remount,size=64G /dev/shm

In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         193G   70G  124G  37% /
tmpfs            64M     0   64M   0% /dev
shm              64G   62M   64G   1% /dev/shm
/dev/nvme0n2    196G   49G  148G  25% /home/jupyter
/dev/nvme0n1p1  193G   70G  124G  37% /usr/local/nvidia/bin


# check models

In [12]:
!ls -lh ~/.cache/huggingface/hub/

total 20K
drwxr-xr-x 6 root root 4.0K Jun  1 06:00 models--openai--clip-vit-base-patch32
drwxr-xr-x 6 root root 4.0K Jun 28 21:14 models--openai--clip-vit-large-patch14
drwxr-xr-x 6 root root 4.0K Jun  1 06:00 models--runwayml--stable-diffusion-v1-5
-rw-r--r-- 1 root root    1 Jun  1 06:00 version.txt
-rw-r--r-- 1 root root    1 Jun  1 06:00 version_diffusers_cache.txt


In [13]:
!ls -lh ~/.cache/torch/hub/checkpoints

total 528M
-rw-r--r-- 1 root root 528M Jun  1 06:04 vgg16-397923af.pth


# run training
### copied HF/hub and torch.hub from last VM to this VM

In [6]:
# 2025-5-20 copied hf hub models to gcs and then copied them back to the target
# gsutil -m cp -r \
#   ~/.cache/huggingface/hub \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/

# on target VM:
# mkdir -p ~/.cache/huggingface/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/hf_hub/hub/* \
#   ~/.cache/huggingface/hub/

# on source vm
# copy torch/hub to the gcs and then to target vm
# gsutil -m cp -r \
#   ~/.cache/torch/hub \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/

# Copying file:///home/jupyter/.cache/torch/hub/checkpoints/vgg16-397923af.pth [Content-Type=application/octet-stream]...
# ==> NOTE: You are uploading one or more large file(s), which would run          
# significantly faster if you enable parallel composite uploads. This
# feature can be enabled by editing the
# "parallel_composite_upload_threshold" value in your .boto
# configuration file. However, note that if you do this large files will
# be uploaded as `composite objects
# <https://cloud.google.com/storage/docs/composite-objects>`_,which
# means that any user who downloads such objects will need to have a
# compiled crcmod installed (see "gsutil help crcmod"). This is because
# without a compiled crcmod, computing checksums on composite objects is
# so slow that gsutil disables downloads of composite objects.

# | [1/1 files][527.8 MiB/527.8 MiB] 100% Done                                    
# Operation completed over 1 objects/527.8 MiB.  

# on target VM:
# mkdir -p ~/.cache/torch/hub
# gsutil -m cp -r \
#   gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/* \
#   ~/.cache/torch/hub/

# Copying gs://merlin-on-vertex-ai-test-central/projects/llm/cv/cubediff/torch_hub/hub/checkpoints/vgg16-397923af.pth...
# - [1/1 files][527.8 MiB/527.8 MiB] 100% Done                                    
# Operation completed over 1 objects/527.8 MiB.             

In [7]:
# 2025-5-20 after copying the models, this should work well.
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
# MAX_LEN  = tokenizer.model_max_length

In [8]:
# MAX_LEN

In [ ]:
import os
os.environ["PYTHONPATH"] = "/home/jupyter/mluser/git/llm-cv-pano-cubediff"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
# Set torch compile backend
os.environ["TORCH_COMPILE_BACKEND"] = "inductor"
os.environ["ACCELERATE_CONFIG_FILE"]="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"
# If set this before importing torch, DeepSpeed’s internal calls to torch.save will automatically use the old writer (no extra temp copy),
# this can save deepspeed checkpoints memory usage and speed up training.
os.environ["TORCH_USE_NEW_ZIPFILE_SERIALIZATION"] = "0"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

In [2]:
ACCELERATE_CONFIG_FILE="/home/jupyter/mluser/git/llm-cv-pano-cubediff/cl/config/accelerate_config.yaml"

In [ ]:
!accelerate launch --config_file $ACCELERATE_CONFIG_FILE train_cubediff.py --cfg ../config/tiny_fullrank.yaml > cubediff_log.txt

ipex flag is deprecated, will be removed in Accelerate v1.10. From 2.7.0, PyTorch has all needed optimizations for Intel CPU and XPU.
Loading pipeline components...: 100%|█████████████| 7/7 [00:03<00:00,  2.18it/s]
